In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
DATASET_PATH = 'Custard Apple dataset'  
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 15

# 1. Load and Preprocess the Dataset
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# 2. Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Train the Model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

# 4. Plot Training Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# 5. Save the Trained Model
model.save("custard_apple_disease_model.keras")

# Save the class labels
with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

print("Model and class labels saved successfully.")

# 6. Predict Function for New Image
def predict_new_image(image_path):
    # Load model
    model = load_model("custard_apple_disease_model.keras")
    
    # Load class indices
    with open("class_indices.json", "r") as f:
        class_indices = json.load(f)
    
    # Reverse the dictionary to get index-to-label mapping
    class_labels = {v: k for k, v in class_indices.items()}
    
    # Load and preprocess image
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_label = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction)

    print(f"Predicted Disease: {predicted_label} (Confidence: {confidence:.2f})")

predict_new_image("testimage4.jpg")


In [1]:
import json
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

IMG_HEIGHT = 128
IMG_WIDTH = 128

def predict_new_image(image_path):
    # Load model
    model = load_model("custard_apple_disease_model.keras")
    
    # Load class indices
    with open("class_indices.json", "r") as f:
        class_indices = json.load(f)
    
    # Reverse the dictionary to get index-to-label mapping
    class_labels = {v: k for k, v in class_indices.items()}
    
    # Load and preprocess image
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_label = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction)

    print(f"Predicted Disease: {predicted_label} (Confidence: {confidence:.2f})")


In [5]:
predict_new_image("downloadbug.jpeg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
Predicted Disease: Mealy Bug (Confidence: 0.97)
